In [1]:
from PickReview.review import *
from PickReview.utils import *

In [2]:
flist = glob.glob("/home/genevieve/research/quakemigrate/cami/dets_fromscamp_h5/20200310*.h5")
filename = flist[0]
ds = pyasdf.ASDFDataSet(filename=filename)
event = ds.events[0].copy()
stream = obspy.Stream()
for id in ds.waveforms.list():
    stream += ds.waveforms[id]["raw_recording"]

picks, _ = fix_picks_ids(event, stream, method="modelled")

print(event)
markers = picks2markers(picks, event=event, phase=True, kinds=(1, 2))
print(event)

#return_tag, markers_out = run_snuffler(stream, events=event, markers=markers, inventory=None)
markers_out = markers
for m in markers_out:
    print(m)

Event:	2020-03-10T06:05:49.352000Z | +50.451, -112.124 | -1.0 ML | automatic

	            resource_id: ResourceIdentifier(id="smi:local/20200310060549888")
	          creation_info: CreationInfo(author='QuakeMigrate', version='1.0.0')
	    preferred_origin_id: ResourceIdentifier(id="smi:local/3d266eec-6313-434e-9e76-c8dc48618ed8")
	 preferred_magnitude_id: ResourceIdentifier(id="smi:local/fde4092d-9daa-4747-a519-c82f1c2e7d27")
	                   ---------
	                  picks: 137 Elements
	             amplitudes: 252 Elements
	                origins: 2 Elements
	             magnitudes: 1 Elements
	     station_magnitudes: 126 Elements
Event:	2020-03-10T06:05:49.352000Z | +50.451, -112.124 | -1.0 ML | automatic

	            resource_id: ResourceIdentifier(id="smi:local/20200310060549888")
	          creation_info: CreationInfo(author='QuakeMigrate', version='1.0.0')
	    preferred_origin_id: ResourceIdentifier(id="smi:local/3d266eec-6313-434e-9e76-c8dc48618ed8")
	 preferred_m

In [3]:
event2 = markers_to_quakeml(markers_out, picks_only=True)
print(event2)

Event:	

	   resource_id: ResourceIdentifier(id="smi:local/e28f7340-7a20-49c0-85a8-fd9c3041aaf1")
	 creation_info: CreationInfo(creation_time=UTCDateTime(2021, 9, 10, 16, 30, 52, 286735))
	          ---------
	         picks: 84 Elements


In [20]:
def write_obs(phase_markers, fname):
    """
    :param phase_markers: list of Pyrocko Phase Markers
    :param fname: output filename for NLLOC OBS phase file
    """
    with open(fname, "w") as outfile:
        for phase_marker in phase_markers:
            if not isinstance(phase_marker, PhaseMarker):
                Logger.info("Skipping marker that isn't a PhaseMarker")
                continue
            station = list(phase_marker.nslc_ids)[0][1]
            channel = list(phase_marker.nslc_ids)[0][3]
            phase_name = phase_marker.get_phasename()  # _phasename

            # Get time and uncertainty
            phase_tmin = phase_marker.tmin
            phase_tmax = phase_marker.tmax
            if phase_tmin == phase_tmax:
                phase_unc = -1
                phase_time = phase_tmin
            else:
                phase_unc = (phase_tmax - phase_tmin) * 0.5
                phase_mid_int = (phase_tmax - phase_tmin) * 0.5
                phase_time = phase_tmin + phase_mid_int
            phase_time_utc = UTCDateTime(REFERENCE_UTC_TIME.utcfromtimestamp(phase_time))

            # First motion polarity
            if phase_marker.get_polarity() == 1:
                polarity = "+"
            elif phase_marker.get_polarity() == -1:
                polarity = "-"
            else:
                polarity = "?"

            # Other defaults
            instrument = "?"
            onset = "?"
            date = phase_time_utc.strftime("%Y%m%d")
            hourmin = phase_time_utc.strftime("%H%M")
            seconds = phase_time_utc.strftime("%S.%f")[:-2]
            error_type = "GAU"
            coda_duration = -1
            amplitude = -1  # Maximum peak-to-peak amplitude
            period = -1  # Period of amplitude reading
            priorwt = 1

            # Write phase to file
            line = f"{station:6s} " \
                   f"{instrument:4s} " \
                   f"{channel:4s} " \
                   f"{onset:1s} " \
                   f"{phase_name:6s} " \
                   f"{polarity:1s} " \
                   f"{date:6s} " \
                   f"{hourmin:4s} " \
                   f"{seconds:7s} " \
                   f"{error_type:4s} " \
                   f"{phase_unc:9.2e} " \
                   f"{coda_duration:9.2e} " \
                   f"{amplitude:9.2e} " \
                   f"{period:9.2e} " \
                   f"{priorwt:9.2e}\n"
            outfile.write(line)
            print("%s\n" % line)

In [21]:
write_obs(markers, "test.obs")

BH001  ?    DPZ  ? P      ? 20200310 0605 49.6357 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH001  ?    DPN  ? S      ? 20200310 0605 49.9728 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH002  ?    DPZ  ? P      ? 20200310 0605 49.6398 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH002  ?    DPN  ? S      ? 20200310 0605 49.9818 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH004  ?    DPZ  ? P      ? 20200310 0605 49.6441 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH004  ?    DPN  ? S      ? 20200310 0605 49.9913 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH005  ?    DPZ  ? P      ? 20200310 0605 49.6441 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH005  ?    DPN  ? S      ? 20200310 0605 49.9913 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH006  ?    DPZ  ? P      ? 20200310 0605 49.6488 GAU  -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00


BH006  ?    DPN  ? S      ? 20200310 